Quest to analyse BLS dataset and usa demographics dataset

In [1]:
import pandas as pd
import boto3
import fsspec
import json
from pandas.core.interchange.dataframe_protocol import DataFrame
from io import BytesIO

In [2]:
bls_path = "s3://s3-quest-bls-dataset-neelima/bls/pr/pr.data.0.Current"
population_usa_path = "s3://s3-quest-bls-dataset-neelima/demographics"
series_id = "PRS30006032"
period = "Q01"
start_year = 2013
end_year = 2018

Data loading to AWS S3

In [3]:
def load_bls_df(bls_path: str):
    bls_df = pd.read_csv(bls_path,sep="\t",dtype=str,compression="gzip")
    return(bls_df)

def load_population_usa_df(population_usa_path: str):
    with fsspec.open(population_usa_path, "r") as f:
        payload = json.load(f)
    population_usa_df = pd.json_normalize(payload["data"])
    return(population_usa_df)

Data cleansing and transformation

In [4]:
def bls_population_data_cleanse(bls_df, population_usa_df):
    bls_df.columns = bls_df.columns.str.strip().str.lower()
    bls_df["series_id"] = bls_df["series_id"].str.strip()
    bls_df["period"] = bls_df["period"].str.strip()
    bls_df["year"] = bls_df["year"].astype(str).str.strip().astype(int)
    bls_df["value"] = bls_df["value"].astype(float)

    # ---- Clean population data ----
    population_usa_df.columns = population_usa_df.columns.str.strip().str.lower()
    population_usa_df["year"] = population_usa_df["year"].astype(str).str.strip().astype(int)
    population_usa_df["population"] = population_usa_df["population"].astype(float)

    return(bls_df, population_usa_df)

Report metrics

In [5]:
def population_usa_stats(pop_df, start_year=2013, end_year=2018):
    population_usa_stats_df = (
        pop_df[(pop_df["year"] >= start_year) & (pop_df["year"] <= end_year)]
        .agg(
            mean_population=("population", "mean"),
            std_population=("population", "std"),
        )
    )
    return(population_usa_stats_df)

In [6]:
def best_year_df(bls_df):
    best_year_df = (
        bls_df
        .groupby(["series_id", "year"], as_index=False)["value"]
        .sum()
        .sort_values(["series_id", "value"], ascending=[True, False])
        .drop_duplicates("series_id")
    )
    return (best_year_df)

In [7]:
def generate_bls_population_report(bls_df, pop_df, series_id, quarter):
    target_df = (bls_df[(bls_df["series_id"] == series_id) & (bls_df["period"] == quarter)]
                 .merge(pop_df[["year", "population"]],on="year",how="left")
                 .sort_values("year"))
    return(target_df)


In [13]:
bls_df = load_bls_df(bls_path)
population_usa_df = load_population_usa_df(population_usa_path)
bls_cleansed_df,population_usa_cleansed_df = bls_population_data_cleanse(bls_df,population_usa_df)
population_usa_stats_df = population_usa_stats(population_usa_cleansed_df,start_year,end_year)
# printing mean and standard deviation
print(f"Mean population: {population_usa_stats_df.loc['mean_population', 'population']:,.0f}")
print(f"Std deviation: {population_usa_stats_df.loc['std_population', 'population']:,.0f}\n")
best_series_id_year_df = best_year_df(bls_cleansed_df)
print("Best year with max value for all quarters for all series_id\n")
print(best_series_id_year_df)
report_df = generate_bls_population_report(bls_cleansed_df,population_usa_cleansed_df,"PRS30006032","Q01")
print("Report to get population for series_id PRS30006032 and quarter Q01\n")
print(report_df[["series_id", "year", "period", "value", "population"]])



Mean population: 322,069,808
Std deviation: 4,158,441

Best year with max value for all quarters for all series_id

        series_id  year    value
27    PRS30006011  2022   20.500
58    PRS30006012  2022   17.100
65    PRS30006013  1998  705.895
108   PRS30006021  2010   17.700
139   PRS30006022  2010   12.400
...           ...   ...      ...
8459  PRS88003192  2002  282.800
8512  PRS88003193  2024  862.564
8541  PRS88003201  2022   38.900
8572  PRS88003202  2022   29.700
8605  PRS88003203  2024  590.619

[282 rows x 3 columns]
Report to get population for series_id PRS30006032 and quarter Q01

      series_id  year period  value   population
0   PRS30006032  1995    Q01    0.0          NaN
1   PRS30006032  1996    Q01   -4.2          NaN
2   PRS30006032  1997    Q01    2.8          NaN
3   PRS30006032  1998    Q01    0.9          NaN
4   PRS30006032  1999    Q01   -4.1          NaN
5   PRS30006032  2000    Q01    0.5          NaN
6   PRS30006032  2001    Q01   -6.3          NaN
7   